# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [42]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
# from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [43]:
# Read the Mapbox API key|
# load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [44]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [45]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    mean_units_per_year = sfo_data.groupby('year').housing_units.mean()
    data_std = mean_units_per_year.std()
    chart_min = min(mean_units_per_year) - data_std
    chart_max = max(mean_units_per_year) + data_std
    
    fig_housing_units = mean_units_per_year.hvplot.bar(
        title="Housing Units in SanFrancisco from 2010 to 2016",
        ylim=(chart_min, chart_max),
        rot=45
    )
    
    return fig_housing_units

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_year_grp = sfo_data.groupby('year')
    avg_gross_rent = sfo_data_year_grp.gross_rent.mean()
    avg_price_sq_ft = sfo_data_year_grp.sale_price_sqr_foot.mean()
    
    agr_plot = avg_gross_rent.hvplot.line(
        title="Average Gross Rent in San Francisco Per Year",
        value_label="Sale price per sq. ft."
    )
    
    return agr_plot

def average_sale_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_year_grp = sfo_data.groupby('year')
    avg_gross_rent = sfo_data_year_grp.gross_rent.mean()
    avg_price_sq_ft = sfo_data_year_grp.sale_price_sqr_foot.mean()
    
    apsf_plot = avg_price_sq_ft.hvplot.line(
        title="Average Sales Price Per Year",
        value_label="Sale price per sq. ft."
    )
    
    return apsf_plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    avg_price_per_neighborhood = sfo_data.groupby(['neighborhood', 'year']).mean()
    line_plot = avg_price_per_neighborhood.hvplot.line(
        title="Average Prices by Neighborhood",
        x='year',
        y='sale_price_sqr_foot',
        ylabel="Sale price per sq. ft.",
        groupby='neighborhood'
    )
    
    return line_plot

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    mean_price_by_neighborhood = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhoods = mean_price_by_neighborhood.sort_values(ascending=False, by='sale_price_sqr_foot').head(10)

    bar_chart = most_expensive_neighborhoods.hvplot.bar(
        y='sale_price_sqr_foot',
        title="Top 10 Most Expensive Neighborhoods",
        rot=45,
        xlabel='Neighborhood',
        ylabel='Avg. Sale Price Per Square Foot'
    )
    
    return bar_chart

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    mean_price_by_neighborhood = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhoods = mean_price_by_neighborhood.sort_values(ascending=False, by='sale_price_sqr_foot').head(10)
    
    pc_chart = px.parallel_coordinates(
        most_expensive_neighborhoods,
        color='sale_price_sqr_foot'
    )
    
    return pc_chart

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    mean_price_by_neighborhood = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhoods = mean_price_by_neighborhood.sort_values(ascending=False, by='sale_price_sqr_foot').head(10)
    
    pc_chart = px.parallel_categories(
        most_expensive_neighborhoods.reset_index(),
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color='sale_price_sqr_foot'
    )
    
    return pc_chart

def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    px.set_mapbox_access_token(mapbox_token)
    
    file_path = Path("Data/neighborhoods_coordinates.csv")
    df_neighborhood_locations = pd.read_csv(file_path)
    df_neighborhood_locations.set_index('Neighborhood', inplace=True)
    df_neighborhood_locations.index.rename('neighborhood', inplace=True)
    df_neighborhood_locations.head()
    df_neighborhood_locations.loc['Bernal Heights']
    
    neighborhood_means = sfo_data.groupby('neighborhood').mean()
    
    # Some neighborhood names in this dataframe have a trailing space, so the join with
    # the dataframe above is left with a few rows with NaN for the Lat and Lon. This
    # fixes that issue.
    neighborhood_means.index = neighborhood_means.index.str.strip(' ')

    neighborhoods = neighborhood_means.join(df_neighborhood_locations)
    
    hood_map = px.scatter_mapbox(
        neighborhoods,
        lat='Lat',
        lon='Lon',
        text=neighborhoods.index,
        zoom=10
    )

    return hood_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [46]:
# YOUR CODE HERE!
col1 = pn.Column(
    "# San Francisco, CA, Housing Data",
    "## Compare data year-to-year",
    housing_units_per_year(),
    "### Compare averages",
    average_gross_rent(),
    average_sale_price()
)

col2 = pn.Column(
    "## Comparisons by neighborhood prices",
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

col3 = pn.Column(
    "## See the relationships between several features",
    parallel_categories(),
    parallel_coordinates()
)

col4 = pn.Column(
    "## Map of San Francisco",
    neighborhood_map()
)

neighborhoods_dashboard = pn.Tabs(
    ("Yearly data", col1),
    ("Prices", col2),
    ("Multi-feature", col3),
    ("Map", col4)
)

## Serve the Panel Dashboard

In [47]:
# main_panel.servable()
neighborhoods_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] HoloViews(Bars)
        [3] Markdown(str)
        [4] HoloViews(Curve)
        [5] HoloViews(Curve)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)